In [1]:
import datetime
import pickle
from pathlib import Path

from tqdm import tqdm

from utils.data_reading.catalogs.isc import ISC_file
from utils.transformations.features_extractor import STFTFeaturesExtractor

/home/plerolland/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
year = 2010

# input files
datasets_yaml = "/home/plerolland/Bureau/dataset.yaml"
detections_file = f"../../../data/T-pick/{year}/log_det"
matchups_raw_file = f"../../../data/T-pick/{year}/matchups"

# output dir
dest = f"/home/plerolland/Bureau/ISC_matchup_check/{year}"

DELTA = datetime.timedelta(seconds=100)
TIME_RES = 0.5
FREQ_RES = 120/128

In [3]:
with open(detections_file, 'rb') as f:
    detections = pickle.load(f)[1]

with open(matchups_raw_file, "rb") as f:
    selected_matchups = pickle.load(f)

STFT_computer = STFTFeaturesExtractor(None, f_min=0, f_max=120, vmin=70, vmax=100)

In [4]:
seen_stations = []

for ID, matchup in tqdm(selected_matchups.items()):
    available_stations = [d[0] for d in detections[ID]]
    for i, station in enumerate(available_stations):
        # try to get station from history if it can be found there
        if station in seen_stations:
            station = seen_stations[seen_stations.index(station)]
        else:
            seen_stations.append(station)

        STFT_computer.manager = station.get_manager()
        STFT_computer.manager.cache_size = 0  # important : enable to avoid loading the whole sound files at each get_segment() call
        STFT_computer.nperseg = round(STFT_computer.manager.sampling_f / FREQ_RES)
        STFT_computer.overlap = 1 - TIME_RES * STFT_computer.manager.sampling_f / STFT_computer.nperseg
        folder = f"{dest}/{station.name}"
        Path(folder).mkdir(parents=True, exist_ok=True)

        det = matchup[station]["time_s"] if station in matchup else -1

        center = detections[ID][i][1]
        start, end = center - DELTA, center + DELTA
        if not Path(f"{folder}/{ID}_{det}.png").exists():
            STFT_computer.save_features(start, end, f"{folder}/{ID}_{det:.2f}.png")

  3%|▎         | 45/1315 [00:25<10:04,  2.10it/s]/home/plerolland/Bureau/OHASISBIO_dataset/src/utils/transformations/features_extractor.py:157: UserWarning: /home/plerolland/Bureau/ISC_matchup_check/2010/H08S1/14683432_-1.00.png is a low contrast image
  skimage.io.imsave(path, spectro.astype(np.uint8))
/home/plerolland/Bureau/OHASISBIO_dataset/src/utils/transformations/features_extractor.py:157: UserWarning: /home/plerolland/Bureau/ISC_matchup_check/2010/H08N1/14683432_-1.00.png is a low contrast image
  skimage.io.imsave(path, spectro.astype(np.uint8))
  5%|▍         | 60/1315 [00:33<12:01,  1.74it/s]/home/plerolland/Bureau/OHASISBIO_dataset/src/utils/transformations/features_extractor.py:157: UserWarning: /home/plerolland/Bureau/ISC_matchup_check/2010/H08S1/17364966_-1.00.png is a low contrast image
  skimage.io.imsave(path, spectro.astype(np.uint8))
/home/plerolland/Bureau/OHASISBIO_dataset/src/utils/transformations/features_extractor.py:157: UserWarning: /home/plerolland/Bureau/IS